In [1]:
# Import libraries
# transformers: Provides access to pre-trained models like CLAP.
from transformers import ClapModel, AutoProcessor

# librosa: Used for audio analysis and loading audio files.
import librosa

# os: Provides a way to interact with the operating system, like navigating directories.
import os

import torch
import pandas as pd
import numpy as np

In [2]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/clap-htsat-unfused")
processor = AutoProcessor.from_pretrained("laion/clap-htsat-unfused")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [3]:
#Small test to print all the parameters of CLAP
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 153492890


In [4]:
#Calculating the size of the data of all the parameters
param_size = 0 # We are using a for loop to add size of all parameters os initialised as 0 at the beginning

for param in model.parameters():
    #In increments we are adding the size of each parameter given by formula
    #param.nelement() is giving total number of elements in each parameter
    #param.element_size() is giving size of element in bytes
    param_size += param.nelement() * param.element_size()

#Buffers are tensors but not learnable parameters but still take up space
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

#Total size adds buffer size and parameter size
#Converted into MB
size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 585.912MB


In [5]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [6]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"  #/content/Exp2/Stimuli/ if we are running experiment 2

#This section adds all the audio waveforms into the list audio_stimuli
for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        print(wav_path)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

/content/Exp1/Stimuli/01_B_CTu_1.wav
/content/Exp1/Stimuli/02_B_CTu_2.wav
/content/Exp1/Stimuli/03_B_Tu_3.wav
/content/Exp1/Stimuli/04_B_Tu_4.wav
/content/Exp1/Stimuli/05_B_CTb_1.wav
/content/Exp1/Stimuli/06_B_CTb_2.wav
/content/Exp1/Stimuli/07_B_BTb_3.wav
/content/Exp1/Stimuli/08_B_TTb_4.wav
/content/Exp1/Stimuli/09_B_Ho_3.wav
/content/Exp1/Stimuli/10_B_Ho_4.wav
/content/Exp1/Stimuli/11_B_Ho_5.wav
/content/Exp1/Stimuli/12_B_Tr_4.wav
/content/Exp1/Stimuli/13_B_PTr_5.wav
/content/Exp1/Stimuli/14_B_PTr_6.wav
/content/Exp1/Stimuli/15_W_CBa_1.wav
/content/Exp1/Stimuli/16_W_CBa_2.wav
/content/Exp1/Stimuli/17_W_Ba_3.wav
/content/Exp1/Stimuli/18_W_Ba_4.wav
/content/Exp1/Stimuli/19_W_BCl_2.wav
/content/Exp1/Stimuli/20_W_BCl_3.wav
/content/Exp1/Stimuli/21_W_ClB_4.wav
/content/Exp1/Stimuli/22_W_ClB_5.wav
/content/Exp1/Stimuli/23_W_ClB_6.wav
/content/Exp1/Stimuli/24_W_Ob_4.wav
/content/Exp1/Stimuli/25_W_Ob_5.wav
/content/Exp1/Stimuli/26_W_EH_4.wav
/content/Exp1/Stimuli/27_W_EH_5.wav
/content/Exp1

In [7]:
#verifcation if we opened correct file
print(audio_stimuli)

[array([-2.6257991e-05, -1.8168332e-05,  4.3254666e-05, ...,
       -1.1775010e-05,  1.0805696e-06, -3.3297738e-08], dtype=float32), array([-2.1216942e-05, -4.7350541e-06,  1.5635709e-05, ...,
        2.8016274e-08, -1.7804325e-08,  6.9179218e-09], dtype=float32), array([-9.8106102e-06,  1.1119668e-05, -3.5152134e-05, ...,
       -3.4414730e-05, -1.3167290e-05,  4.1392400e-06], dtype=float32), array([-2.9096404e-06, -1.3023089e-05, -4.7573376e-06, ...,
       -3.4261415e-05, -2.2727640e-05,  2.1467149e-06], dtype=float32), array([ 2.7754413e-06, -7.0806864e-06,  1.1189784e-05, ...,
       -1.2227122e-06,  7.4793968e-07, -2.8134855e-07], dtype=float32), array([-6.7172846e-07,  5.4504471e-06, -5.0830917e-05, ...,
       -1.6670765e-06,  1.9083664e-06,  0.0000000e+00], dtype=float32), array([ 1.6268901e-05, -3.3643319e-05,  2.5531041e-05, ...,
       -3.3158278e-05, -2.5886366e-05,  7.7379474e-07], dtype=float32), array([ 6.1509754e-06,  2.7266901e-06, -4.9879025e-05, ...,
       -1.23645

In [8]:
#Processes the audio waveform in the audio_stimuli list
#The process involves converting the audio files into tensors (As PyTorch deals with Tensors)
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)

#Inputs the tensors into the CLAP model to extract the audio embeddings
#These embeddings are numerical representations
audio_embeddings = model.get_audio_features(**inputs)

In [9]:
#Returning dimensions of the tensors
#The 59 is how many audio files there are
#The 512 is the size of any audio embedding
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [10]:
#This is generating the prompts for each section
#We have 2 sections of discrete and dimensional
#Each section has 2 subsections of perceived and induced
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake']


In [11]:
#Combines all the generated text prompts from previous cell into one list
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# NOTE: currently using only dimensional_captions_induced
tag_inputs = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds = model.get_text_features(**tag_inputs)

# Generate outputs

In [12]:
#Computes the similaity matrix between each audio clip and text tag
#Computes this using dot product between the audio embeddings and text embeddings
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 5])


## Load csv files and extract related columns

In [13]:
IDisc_happiness_path = '/content/Exp1/Data/PDisc'
IDisc_happiness_response_dfs = []

for file in os.listdir(IDisc_happiness_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_happiness_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['happiness']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_happiness_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_happiness_response_dfs:
    master_human_responses_df_happiness = pd.concat(IDisc_happiness_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_happiness.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_happiness.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in PDisc_happiness_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   happiness
0       1.47
1       1.00
2       5.19
3       1.00
4       1.00



In [14]:
IDisc_sadness_path = '/content/Exp1/Data/PDisc'
IDisc_sadness_response_dfs = []

for file in os.listdir(IDisc_sadness_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_sadness_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['sadness']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_sadness_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_sadness_response_dfs:
    master_human_responses_df_sadness = pd.concat(IDisc_sadness_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_sadness.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_sadness.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in PDisc_sadness_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   sadness
0     6.92
1     6.50
2     5.81
3     6.92
4     3.53



In [15]:
IDisc_anger_path = '/content/Exp1/Data/PDisc'
IDisc_anger_response_dfs = []

for file in os.listdir(IDisc_anger_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_anger_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['anger']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_anger_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_anger_response_dfs:
    master_human_responses_df_anger = pd.concat(IDisc_anger_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_anger.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_anger.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in PDisc_anger_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   anger
0   4.96
1   3.19
2   1.55
3   1.27
4   4.97



In [16]:
IDisc_tenderness_path = '/content/Exp1/Data/PDisc'
IDisc_tenderness_response_dfs = []

for file in os.listdir(IDisc_tenderness_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_tenderness_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['tenderness']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_tenderness_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_tenderness_response_dfs:
    master_human_responses_df_tenderness = pd.concat(IDisc_tenderness_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_tenderness.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_tenderness.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in PDisc_tenderness_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   tenderness
0        2.33
1        1.78
2        3.23
3        2.14
4        1.00



In [17]:
IDisc_fear_path = '/content/Exp1/Data/PDisc'
IDisc_fear_response_dfs = []

for file in os.listdir(IDisc_fear_path):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_fear_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['fear']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_fear_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_fear_response_dfs:
    master_human_responses_df_fear = pd.concat(IDisc_fear_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_fear.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_fear.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in PDisc_fear_path.")


Master human responses DataFrame shape: (3835, 1)

Master human responses (first 5 rows):
   fear
0  4.09
1  2.76
2  2.53
3  1.66
4  2.10



# Prepare features X and targets y

In [18]:
from sklearn.model_selection import train_test_split


num_participants_happiness = len(IDisc_happiness_response_dfs)
if master_human_responses_df_happiness.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_happiness = []
for _ in range(num_participants_happiness):
    X_list_happiness.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_happiness = np.array(X_list_happiness)

# Extract y from the concatenated DataFrame
y_happiness = master_human_responses_df_happiness[['happiness']].values

print(f"Shape of X (features) after implicit alignment: {X_happiness.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_happiness.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_happiness.shape[0] != y_happiness.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_happiness = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_happiness = 1 - TEST_PERCENTAGE_happiness  # Adjust as needed

X_train_happiness, X_test_happiness, y_train_happiness, y_test_happiness = train_test_split(
    X_happiness, y_happiness, test_size=TEST_PERCENTAGE_happiness, random_state=42
)

print(f"Training set size (X_train_happiness, y_train_happiness): {X_train_happiness.shape}, {y_train_happiness.shape}")
print(f"Testing set size (X_test_happiness, y_test_happiness): {X_test_happiness.shape}, {y_test_happiness.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_happiness, y_train_happiness): (3068, 512), (3068, 1)
Testing set size (X_test_happiness, y_test_happiness): (767, 512), (767, 1)



In [19]:
from sklearn.model_selection import train_test_split


num_participants_sadness = len(IDisc_sadness_response_dfs)
if master_human_responses_df_sadness.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_sadness = []
for _ in range(num_participants_sadness):
    X_list_sadness.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_sadness = np.array(X_list_sadness)

# Extract y from the concatenated DataFrame
y_sadness = master_human_responses_df_sadness[['sadness']].values

print(f"Shape of X (features) after implicit alignment: {X_sadness.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_sadness.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_sadness.shape[0] != y_sadness.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_sadness = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_sadness = 1 - TEST_PERCENTAGE_sadness  # Adjust as needed

X_train_sadness, X_test_sadness, y_train_sadness, y_test_sadness = train_test_split(
    X_sadness, y_sadness, test_size=TEST_PERCENTAGE_sadness, random_state=42
)

print(f"Training set size (X_train_sadness, y_train_sadness): {X_train_sadness.shape}, {y_train_sadness.shape}")
print(f"Testing set size (X_test_sadness, y_test_sadness): {X_test_sadness.shape}, {y_test_sadness.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_sadness, y_train_sadness): (3068, 512), (3068, 1)
Testing set size (X_test_sadness, y_test_sadness): (767, 512), (767, 1)



In [20]:
from sklearn.model_selection import train_test_split


num_participants_anger = len(IDisc_anger_response_dfs)
if master_human_responses_df_anger.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_anger = []
for _ in range(num_participants_anger):
    X_list_anger.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_anger = np.array(X_list_anger)

# Extract y from the concatenated DataFrame
y_anger = master_human_responses_df_anger[['anger']].values

print(f"Shape of X (features) after implicit alignment: {X_anger.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_anger.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_anger.shape[0] != y_anger.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_anger = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_anger = 1 - TEST_PERCENTAGE_anger  # Adjust as needed

X_train_anger, X_test_anger, y_train_anger, y_test_anger = train_test_split(
    X_anger, y_anger, test_size=TEST_PERCENTAGE_anger, random_state=42
)

print(f"Training set size (X_train_anger, y_train_anger): {X_train_anger.shape}, {y_train_anger.shape}")
print(f"Testing set size (X_test_anger, y_test_anger): {X_test_anger.shape}, {y_test_anger.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_anger, y_train_anger): (3068, 512), (3068, 1)
Testing set size (X_test_anger, y_test_anger): (767, 512), (767, 1)



In [21]:
from sklearn.model_selection import train_test_split


num_participants_tenderness = len(IDisc_tenderness_response_dfs)
if master_human_responses_df_tenderness.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_tenderness = []
for _ in range(num_participants_tenderness):
    X_list_tenderness.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_tenderness = np.array(X_list_tenderness)

# Extract y from the concatenated DataFrame
y_tenderness = master_human_responses_df_tenderness[['tenderness']].values

print(f"Shape of X (features) after implicit alignment: {X_tenderness.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_tenderness.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_tenderness.shape[0] != y_tenderness.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_tenderness = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_tenderness = 1 - TEST_PERCENTAGE_tenderness  # Adjust as needed

X_train_tenderness, X_test_tenderness, y_train_tenderness, y_test_tenderness = train_test_split(
    X_tenderness, y_tenderness, test_size=TEST_PERCENTAGE_tenderness, random_state=42
)

print(f"Training set size (X_train_tenderness, y_train_tenderness): {X_train_tenderness.shape}, {y_train_tenderness.shape}")
print(f"Testing set size (X_test_tenderness, y_test_tenderness): {X_test_tenderness.shape}, {y_test_tenderness.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_tenderness, y_train_tenderness): (3068, 512), (3068, 1)
Testing set size (X_test_tenderness, y_test_tenderness): (767, 512), (767, 1)



In [22]:
from sklearn.model_selection import train_test_split


num_participants_fear = len(IDisc_fear_response_dfs)
if master_human_responses_df_fear.shape[0] % len(audio_stimuli) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list_fear = []
for _ in range(num_participants_fear):
    X_list_fear.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_fear = np.array(X_list_fear)

# Extract y from the concatenated DataFrame
y_fear = master_human_responses_df_fear[['fear']].values

print(f"Shape of X (features) after implicit alignment: {X_fear.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_fear.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_fear.shape[0] != y_fear.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---

TEST_PERCENTAGE_fear = 0.2  # Adjust as needed
TRAIN_PERCENTAGE_fear = 1 - TEST_PERCENTAGE_fear  # Adjust as needed

X_train_fear, X_test_fear, y_train_fear, y_test_fear = train_test_split(
    X_fear, y_fear, test_size=TEST_PERCENTAGE_fear, random_state=42
)

print(f"Training set size (X_train_fear, y_train_fear): {X_train_fear.shape}, {y_train_fear.shape}")
print(f"Testing set size (X_test_fear, y_test_fear): {X_test_fear.shape}, {y_test_fear.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 1)

Training set size (X_train_fear, y_train_fear): (3068, 512), (3068, 1)
Testing set size (X_test_fear, y_test_fear): (767, 512), (767, 1)



# Train regression head (=MLP, a few projection layers)

In [23]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_happiness = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_happiness training...")
mlp_regressor_happiness.fit(X_train_happiness, y_train_happiness)
print("\nMLP Regressor_happiness training complete.")


Starting MLP Regressor_happiness training...
Iteration 1, loss = 5.73580943
Validation score: -0.807655
Iteration 2, loss = 3.42583798
Validation score: 0.010081
Iteration 3, loss = 2.32753941
Validation score: 0.097252
Iteration 4, loss = 2.07885471
Validation score: 0.133187
Iteration 5, loss = 2.00986599
Validation score: 0.130749


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 6, loss = 1.98654793
Validation score: 0.136966
Iteration 7, loss = 1.96904192
Validation score: 0.141796
Iteration 8, loss = 1.95966612
Validation score: 0.138967
Iteration 9, loss = 1.95071235
Validation score: 0.141695
Iteration 10, loss = 1.94401848
Validation score: 0.141794
Iteration 11, loss = 1.94011214
Validation score: 0.143296
Iteration 12, loss = 1.93812797
Validation score: 0.138840
Iteration 13, loss = 1.93394846
Validation score: 0.139119
Iteration 14, loss = 1.93429092
Validation score: 0.142912
Iteration 15, loss = 1.92774100
Validation score: 0.135705
Iteration 16, loss = 1.92277569
Validation score: 0.142840
Iteration 17, loss = 1.92302633
Validation score: 0.137038
Iteration 18, loss = 1.91994782
Validation score: 0.140522
Iteration 19, loss = 1.91715611
Validation score: 0.140657
Iteration 20, loss = 1.91456751
Validation score: 0.137329
Iteration 21, loss = 1.91108961
Validation score: 0.136303
Iteration 22, loss = 1.91413814
Validation score: 0.137467
I

In [24]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_sadness = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_sadness training...")
mlp_regressor_sadness.fit(X_train_sadness, y_train_sadness)
print("\nMLP Regressor_sadness training complete.")


Starting MLP Regressor_sadness training...
Iteration 1, loss = 8.90503887
Validation score: -1.189987
Iteration 2, loss = 5.30092967
Validation score: -0.034599
Iteration 3, loss = 2.88694319
Validation score: 0.074394
Iteration 4, loss = 2.73965739
Validation score: 0.154967


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 5, loss = 2.63011289
Validation score: 0.154118
Iteration 6, loss = 2.61106155
Validation score: 0.153289
Iteration 7, loss = 2.59873123
Validation score: 0.154216
Iteration 8, loss = 2.59391732
Validation score: 0.153982
Iteration 9, loss = 2.59099368
Validation score: 0.154017
Iteration 10, loss = 2.58498878
Validation score: 0.154933
Iteration 11, loss = 2.58104803
Validation score: 0.157139
Iteration 12, loss = 2.57848355
Validation score: 0.158941
Iteration 13, loss = 2.57562954
Validation score: 0.157718
Iteration 14, loss = 2.57430169
Validation score: 0.158415
Iteration 15, loss = 2.57064998
Validation score: 0.160359
Iteration 16, loss = 2.56918964
Validation score: 0.159555
Iteration 17, loss = 2.56664800
Validation score: 0.162549
Iteration 18, loss = 2.56970095
Validation score: 0.163913
Iteration 19, loss = 2.56939237
Validation score: 0.163302
Iteration 20, loss = 2.56306411
Validation score: 0.161328
Iteration 21, loss = 2.56608817
Validation score: 0.165815
It

In [25]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_anger = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_anger training...")
mlp_regressor_anger.fit(X_train_anger, y_train_anger)
print("\nMLP Regressor_anger training complete.")


Starting MLP Regressor_anger training...
Iteration 1, loss = 6.47466141
Validation score: -0.728076
Iteration 2, loss = 3.86713953
Validation score: 0.055588
Iteration 3, loss = 2.68978406
Validation score: 0.047352
Iteration 4, loss = 2.52569558
Validation score: 0.142925
Iteration 5, loss = 2.42894211
Validation score: 0.161071


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 6, loss = 2.34856524
Validation score: 0.183839
Iteration 7, loss = 2.29984301
Validation score: 0.191266
Iteration 8, loss = 2.26517554
Validation score: 0.204717
Iteration 9, loss = 2.24029243
Validation score: 0.210822
Iteration 10, loss = 2.22508413
Validation score: 0.212201
Iteration 11, loss = 2.21025740
Validation score: 0.216462
Iteration 12, loss = 2.20427397
Validation score: 0.216687
Iteration 13, loss = 2.19250662
Validation score: 0.216981
Iteration 14, loss = 2.18793690
Validation score: 0.221753
Iteration 15, loss = 2.18217542
Validation score: 0.220160
Iteration 16, loss = 2.18094814
Validation score: 0.221203
Iteration 17, loss = 2.17407562
Validation score: 0.222641
Iteration 18, loss = 2.17222558
Validation score: 0.215459
Iteration 19, loss = 2.17346732
Validation score: 0.224274
Iteration 20, loss = 2.17200885
Validation score: 0.212859
Iteration 21, loss = 2.16481605
Validation score: 0.223344
Iteration 22, loss = 2.16608948
Validation score: 0.219459
I

In [26]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_tenderness = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_tenderness training...")
mlp_regressor_tenderness.fit(X_train_tenderness, y_train_tenderness)
print("\nMLP Regressor_tenderness training complete.")


Starting MLP Regressor_tenderness training...
Iteration 1, loss = 7.10636799
Validation score: -1.013812
Iteration 2, loss = 4.35155470
Validation score: -0.099533
Iteration 3, loss = 2.83043216
Validation score: -0.012657
Iteration 4, loss = 2.59398215
Validation score: 0.011742
Iteration 5, loss = 2.52109998
Validation score: 0.016471


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 6, loss = 2.48641107
Validation score: 0.031541
Iteration 7, loss = 2.46350559
Validation score: 0.042676
Iteration 8, loss = 2.44717986
Validation score: 0.047728
Iteration 9, loss = 2.44281407
Validation score: 0.048419
Iteration 10, loss = 2.42815355
Validation score: 0.047108
Iteration 11, loss = 2.42486478
Validation score: 0.047669
Iteration 12, loss = 2.42043677
Validation score: 0.046507
Iteration 13, loss = 2.41937279
Validation score: 0.050918
Iteration 14, loss = 2.41780017
Validation score: 0.049443
Iteration 15, loss = 2.42777465
Validation score: 0.049586
Iteration 16, loss = 2.40948432
Validation score: 0.047180
Iteration 17, loss = 2.41188170
Validation score: 0.048829
Iteration 18, loss = 2.41611536
Validation score: 0.049237
Iteration 19, loss = 2.41643245
Validation score: 0.048525
Iteration 20, loss = 2.40705291
Validation score: 0.048762
Iteration 21, loss = 2.40265953
Validation score: 0.045830
Iteration 22, loss = 2.40375787
Validation score: 0.046262
I

In [27]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_fear = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42, #Determines random number generation for weights and bias
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor_fear training...")
mlp_regressor_fear.fit(X_train_fear, y_train_fear)
print("\nMLP Regressor_fear training complete.")


Starting MLP Regressor_fear training...
Iteration 1, loss = 7.31898207
Validation score: -0.963142
Iteration 2, loss = 4.41713891
Validation score: 0.019881
Iteration 3, loss = 2.84663970
Validation score: 0.060667
Iteration 4, loss = 2.73001111
Validation score: 0.132527
Iteration 5, loss = 2.61725638
Validation score: 0.153823


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1650: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 6, loss = 2.54352075
Validation score: 0.165179
Iteration 7, loss = 2.49579169
Validation score: 0.175824
Iteration 8, loss = 2.45878670
Validation score: 0.187224
Iteration 9, loss = 2.42882040
Validation score: 0.194394
Iteration 10, loss = 2.40560194
Validation score: 0.200988
Iteration 11, loss = 2.39243975
Validation score: 0.206159
Iteration 12, loss = 2.37340612
Validation score: 0.208621
Iteration 13, loss = 2.36043808
Validation score: 0.214220
Iteration 14, loss = 2.35816455
Validation score: 0.215965
Iteration 15, loss = 2.35399068
Validation score: 0.216958
Iteration 16, loss = 2.34381436
Validation score: 0.217691
Iteration 17, loss = 2.34060092
Validation score: 0.219914
Iteration 18, loss = 2.33771115
Validation score: 0.218894
Iteration 19, loss = 2.33267277
Validation score: 0.220956
Iteration 20, loss = 2.33575138
Validation score: 0.223380
Iteration 21, loss = 2.33039732
Validation score: 0.222123
Iteration 22, loss = 2.33010302
Validation score: 0.222025
I

# Evaluate

In [28]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for PDisc – happiness")
print("______________________________________________")

# Predict and reshape if needed
y_pred_happiness = mlp_regressor_happiness.predict(X_test_happiness)
if y_pred_happiness.ndim > 1:
    y_pred_happiness = y_pred_happiness.flatten()
if y_test_happiness.ndim > 1:
    y_test_happiness = y_test_happiness.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_happiness.shape}")
print(f"First 5 actual values (y_test):\n{y_test_happiness[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_happiness[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_happiness * 100)}/{int(TEST_PERCENTAGE_happiness * 100)}\n")

# Evaluation Metrics
mae_happiness = mean_absolute_error(y_test_happiness, y_pred_happiness)
rmse_happiness = np.sqrt(mean_squared_error(y_test_happiness, y_pred_happiness))
r2_happiness = r2_score(y_test_happiness, y_pred_happiness)

print(f"Mean Absolute Error (MAE): {mae_happiness:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_happiness:.4f}")
print(f"R-squared (R²): {r2_happiness:.4f}")

# Pearson Correlation
if np.std(y_test_happiness) > 1e-6 and np.std(y_pred_happiness) > 1e-6:
    corr_happiness, _ = pearsonr(y_test_happiness, y_pred_happiness)
    print(f"Pearson Correlation (happiness): {corr_happiness:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_happiness_safe = np.clip(y_test_happiness, a_min=1e-6, a_max=None)
mape_happiness = np.mean(np.mean(np.abs((y_test_happiness - y_pred_happiness) / y_test_happiness_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_happiness:.2f}%")

Results for PDisc – happiness
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[1.91]
 [1.  ]
 [1.  ]
 [4.96]
 [3.37]]
First 5 predicted values (y_pred):
[[1.9072641]
 [1.4473593]
 [2.443255 ]
 [2.2815006]
 [2.9020681]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.5142
Root Mean Squared Error (RMSE): 1.9296
R-squared (R²): 0.2621
Pearson Correlation (happiness): 0.5149
Mean Absolute Percentage Error (MAPE): 75.94%


In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for PDisc – sadness")
print("______________________________________________")

# Predict and reshape if needed
y_pred_sadness = mlp_regressor_sadness.predict(X_test_sadness)
if y_pred_sadness.ndim > 1:
    y_pred_sadness = y_pred_sadness.flatten()
if y_test_sadness.ndim > 1:
    y_test_sadness = y_test_sadness.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_sadness.shape}")
print(f"First 5 actual values (y_test):\n{y_test_sadness[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_sadness[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_sadness * 100)}/{int(TEST_PERCENTAGE_sadness * 100)}\n")

# Evaluation Metrics
mae_sadness = mean_absolute_error(y_test_sadness, y_pred_sadness)
rmse_sadness = np.sqrt(mean_squared_error(y_test_sadness, y_pred_sadness))
r2_sadness = r2_score(y_test_sadness, y_pred_sadness)

print(f"Mean Absolute Error (MAE): {mae_sadness:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_sadness:.4f}")
print(f"R-squared (R²): {r2_sadness:.4f}")

# Pearson Correlation
if np.std(y_test_sadness) > 1e-6 and np.std(y_pred_sadness) > 1e-6:
    corr_sadness, _ = pearsonr(y_test_sadness, y_pred_sadness)
    print(f"Pearson Correlation (sadness): {corr_sadness:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_sadness_safe = np.clip(y_test_sadness, a_min=1e-6, a_max=None)
mape_sadness = np.mean(np.mean(np.abs((y_test_sadness - y_pred_sadness) / y_test_sadness_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_sadness:.2f}%")

Results for PDisc – sadness
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[2.07]
 [4.06]
 [6.38]
 [1.78]
 [6.96]]
First 5 predicted values (y_pred):
[[5.5252967]
 [4.759757 ]
 [3.9122984]
 [4.089054 ]
 [4.156491 ]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.9367
Root Mean Squared Error (RMSE): 2.3117
R-squared (R²): 0.1070
Pearson Correlation (sadness): 0.3411
Mean Absolute Percentage Error (MAPE): 92.35%


In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for PDisc – anger")
print("______________________________________________")

# Predict and reshape if needed
y_pred_anger = mlp_regressor_anger.predict(X_test_anger)
if y_pred_anger.ndim > 1:
    y_pred_anger = y_pred_anger.flatten()
if y_test_anger.ndim > 1:
    y_test_anger = y_test_anger.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_anger.shape}")
print(f"First 5 actual values (y_test):\n{y_test_anger[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_anger[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_anger * 100)}/{int(TEST_PERCENTAGE_anger * 100)}\n")

# Evaluation Metrics
mae_anger = mean_absolute_error(y_test_anger, y_pred_anger)
rmse_anger = np.sqrt(mean_squared_error(y_test_anger, y_pred_anger))
r2_anger = r2_score(y_test_anger, y_pred_anger)

print(f"Mean Absolute Error (MAE): {mae_anger:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_anger:.4f}")
print(f"R-squared (R²): {r2_anger:.4f}")

# Pearson Correlation
if np.std(y_test_anger) > 1e-6 and np.std(y_pred_anger) > 1e-6:
    corr_anger, _ = pearsonr(y_test_anger, y_pred_anger)
    print(f"Pearson Correlation (anger): {corr_anger:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_anger_safe = np.clip(y_test_anger, a_min=1e-6, a_max=None)
mape_anger = np.mean(np.mean(np.abs((y_test_anger - y_pred_anger) / y_test_anger_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_anger:.2f}%")

Results for PDisc – anger
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[5.69]
 [3.65]
 [1.23]
 [2.97]
 [5.09]]
First 5 predicted values (y_pred):
[[2.8377845]
 [5.0170693]
 [3.631029 ]
 [4.5770807]
 [2.29879  ]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.7040
Root Mean Squared Error (RMSE): 2.1264
R-squared (R²): 0.2429
Pearson Correlation (anger): 0.4935
Mean Absolute Percentage Error (MAPE): 85.24%


In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for PDisc – tenderness")
print("______________________________________________")

# Predict and reshape if needed
y_pred_tenderness = mlp_regressor_tenderness.predict(X_test_tenderness)
if y_pred_tenderness.ndim > 1:
    y_pred_tenderness = y_pred_tenderness.flatten()
if y_test_tenderness.ndim > 1:
    y_test_tenderness = y_test_tenderness.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_tenderness.shape}")
print(f"First 5 actual values (y_test):\n{y_test_tenderness[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_tenderness[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_tenderness * 100)}/{int(TEST_PERCENTAGE_tenderness * 100)}\n")

# Evaluation Metrics
mae_tenderness = mean_absolute_error(y_test_tenderness, y_pred_tenderness)
rmse_tenderness = np.sqrt(mean_squared_error(y_test_tenderness, y_pred_tenderness))
r2_tenderness = r2_score(y_test_tenderness, y_pred_tenderness)

print(f"Mean Absolute Error (MAE): {mae_tenderness:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_tenderness:.4f}")
print(f"R-squared (R²): {r2_tenderness:.4f}")

# Pearson Correlation
if np.std(y_test_tenderness) > 1e-6 and np.std(y_pred_tenderness) > 1e-6:
    corr_tenderness, _ = pearsonr(y_test_tenderness, y_pred_tenderness)
    print(f"Pearson Correlation (tenderness): {corr_tenderness:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_tenderness_safe = np.clip(y_test_tenderness, a_min=1e-6, a_max=None)
mape_tenderness = np.mean(np.mean(np.abs((y_test_tenderness - y_pred_tenderness) / y_test_tenderness_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_tenderness:.2f}%")

Results for PDisc – tenderness
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[2.2 ]
 [1.91]
 [1.  ]
 [1.  ]
 [4.77]]
First 5 predicted values (y_pred):
[[2.6218917]
 [2.2208643]
 [2.9717832]
 [2.3886743]
 [3.8589895]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.7445
Root Mean Squared Error (RMSE): 2.1174
R-squared (R²): 0.0924
Pearson Correlation (tenderness): 0.3271
Mean Absolute Percentage Error (MAPE): 91.08%


In [32]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr
import numpy as np

print("Results for PDisc – fear")
print("______________________________________________")

# Predict and reshape if needed
y_pred_fear = mlp_regressor_fear.predict(X_test_fear)
if y_pred_fear.ndim > 1:
    y_pred_fear = y_pred_fear.flatten()
if y_test_fear.ndim > 1:
    y_test_fear = y_test_fear.flatten()

print(f"\nShape of predictions (y_pred): {y_pred_fear.shape}")
print(f"First 5 actual values (y_test):\n{y_test_fear[:5].reshape(-1, 1)}")
print(f"First 5 predicted values (y_pred):\n{y_pred_fear[:5].reshape(-1, 1)}")

print(f"Training/ Test split: {int(TRAIN_PERCENTAGE_fear * 100)}/{int(TEST_PERCENTAGE_fear * 100)}\n")

# Evaluation Metrics
mae_fear = mean_absolute_error(y_test_fear, y_pred_fear)
rmse_fear = np.sqrt(mean_squared_error(y_test_fear, y_pred_fear))
r2_fear = r2_score(y_test_fear, y_pred_fear)

print(f"Mean Absolute Error (MAE): {mae_fear:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_fear:.4f}")
print(f"R-squared (R²): {r2_fear:.4f}")

# Pearson Correlation
if np.std(y_test_fear) > 1e-6 and np.std(y_pred_fear) > 1e-6:
    corr_fear, _ = pearsonr(y_test_fear, y_pred_fear)
    print(f"Pearson Correlation (Fear): {corr_fear:.4f}")
else:
    print("Pearson Correlation: Cannot calculate (insufficient variance)")

#Mean Absolute Percentage Error
y_test_fear_safe = np.clip(y_test_fear, a_min=1e-6, a_max=None)
mape_fear = np.mean(np.mean(np.abs((y_test_fear - y_pred_fear) / y_test_fear_safe)) * 100)
print(f"Mean Absolute Percentage Error (MAPE): {mape_fear:.2f}%")

Results for PDisc – fear
______________________________________________

Shape of predictions (y_pred): (767,)
First 5 actual values (y_test):
[[1.94]
 [1.  ]
 [6.95]
 [1.  ]
 [6.99]]
First 5 predicted values (y_pred):
[[2.8657591]
 [3.9985194]
 [4.2280087]
 [5.1000733]
 [3.2189002]]
Training/ Test split: 80/20

Mean Absolute Error (MAE): 1.8730
Root Mean Squared Error (RMSE): 2.2693
R-squared (R²): 0.1873
Pearson Correlation (Fear): 0.4343
Mean Absolute Percentage Error (MAPE): 90.31%


In [33]:
# Organize metrics into a dictionary
print("Results for Discrete Percieved (PDisc)")
print("______________________________________________")
data = {
    "Dimension": ["happiness", "sadness", "anger","tenderness", "fear"],
    "MAE": [mae_happiness, mae_sadness, mae_anger, mae_tenderness,mae_fear],
    "RMSE": [rmse_happiness, rmse_sadness, rmse_anger, rmse_tenderness, rmse_fear],
    "R²": [r2_happiness, r2_sadness, r2_anger, r2_tenderness, r2_fear],
    "Pearson Corr.": [corr_happiness, corr_sadness, corr_anger, corr_tenderness, corr_fear],
    "MAPE": [mape_happiness, mape_sadness, mape_anger, mape_tenderness, mape_fear]
}

# Create DataFrame
df_metrics = pd.DataFrame(data)

# Add average row
avg_row = {
    "Dimension": "Average",
    "MAE": df_metrics["MAE"].mean(),
    "RMSE": df_metrics["RMSE"].mean(),
    "R²": df_metrics["R²"].mean(),
    "Pearson Corr.": df_metrics["Pearson Corr."].mean(),
    "MAPE": df_metrics["MAPE"].mean()
}
df_metrics = pd.concat([df_metrics, pd.DataFrame([avg_row])], ignore_index=True)


# Display
print(df_metrics.round(4))


Results for Discrete Percieved (PDisc)
______________________________________________
    Dimension     MAE    RMSE      R²  Pearson Corr.     MAPE
0   happiness  1.5142  1.9296  0.2621         0.5149  75.9402
1     sadness  1.9367  2.3117  0.1070         0.3411  92.3534
2       anger  1.7040  2.1264  0.2429         0.4935  85.2388
3  tenderness  1.7445  2.1174  0.0924         0.3271  91.0834
4        fear  1.8730  2.2693  0.1873         0.4343  90.3091
5     Average  1.7545  2.1509  0.1783         0.4222  86.9850
